# 🛒 Retail Promo Analysis Dashboard

## ✅ Step 0: Authenticate and Initialize BigQuery

## 🔐 Step 1: Authenticate & Connect to BigQuery

In [ ]:

from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
client = bigquery.Client(project="retail-1-462621")


## ✅ Step 1: Total Weekly Sales Trend

## 📊 Step 2: Load & Explore Sales Data

In [ ]:

query = '''
    SELECT
        Date,
        SUM(Weekly_Sales) AS Total_Sales
    FROM `retail-1-462621.walmart_data_1.walmart_sales`
    GROUP BY Date
    ORDER BY Date
'''
df_weekly_sales = client.query(query).to_dataframe()
df_weekly_sales['Date'] = pd.to_datetime(df_weekly_sales['Date'])
print(df_weekly_sales.head())


## ✅ Step 2: Top Performing Stores

## 🏪 Step 3: Identify Top Performing Stores

In [ ]:

query = '''
    SELECT
        Store,
        SUM(Weekly_Sales) AS Total_Sales
    FROM `retail-1-462621.walmart_data_1.walmart_sales`
    GROUP BY Store
    ORDER BY Total_Sales DESC
    LIMIT 10
'''
df_top_stores = client.query(query).to_dataframe()
print(df_top_stores)


## ✅ Step 3: Promo vs Non-Promo Sales Summary

## 🎯 Step 4: Promo vs Non-Promo Sales Analysis

In [ ]:

query = '''
    SELECT
        Holiday_Flag,
        AVG(Weekly_Sales) AS Avg_Sales,
        COUNT(*) AS Record_Count
    FROM `retail-1-462621.walmart_data_1.walmart_sales`
    GROUP BY Holiday_Flag
'''
df_promo_summary = client.query(query).to_dataframe()
print(df_promo_summary)


## ✅ Step 4: Store-Level Weekly Sales (Filtered View)

## 📈 Step 5: Store-Level Weekly Sales Trends

In [ ]:

query = '''
    SELECT
        Date,
        Store,
        SUM(Weekly_Sales) AS Weekly_Sales
    FROM `retail-1-462621.walmart_data_1.walmart_sales`
    GROUP BY Date, Store
    ORDER BY Date, Store
'''
df_store_sales = client.query(query).to_dataframe()
df_store_sales['Date'] = pd.to_datetime(df_store_sales['Date'])
filtered = df_store_sales[df_store_sales['Store'].isin([1, 2, 3])]
print(filtered.head())


## ✅ Step 5: State-Wise Sales Totals (from merged data)

## 🌐 Step 6: Merge with Store Location Data

In [ ]:

# Assuming df_map is created from merging df_sales with store location data
state_sales = df_map.groupby("state")["Total_Sales"].sum().sort_values(ascending=False).reset_index()
print(state_sales.head())


## ✅ Step 6: Visualize Promo Impact

## 📍 Step 7: State-Wise Sales Summary

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(data=df_promo_summary, x="Holiday_Flag", y="Avg_Sales")
plt.title("Average Weekly Sales: Holiday vs Non-Holiday")
plt.xlabel("Holiday Flag (1 = Promo Week)")
plt.ylabel("Average Sales")
plt.show()


## ✅ Step 7: Weekly Sales Line Chart

## 🔮 Step 8: Forecast Weekly Sales using Prophet

In [ ]:

plt.figure(figsize=(12, 5))
plt.plot(df_weekly_sales['Date'], df_weekly_sales['Total_Sales'], marker='o')
plt.title("Total Weekly Sales Across the U.S.")
plt.xlabel("Date")
plt.ylabel("Total Sales")
plt.grid(True)
plt.tight_layout()
plt.show()


## ✅ Step 8: Geographic Store Sales Map

## 🗺️ Step 9: Map of Store Locations

In [ ]:

import plotly.express as px

fig = px.scatter_mapbox(
    df_map,
    lat="latitude",
    lon="longitude",
    size="Total_Sales",
    color="Total_Sales",
    hover_name="name",
    mapbox_style="carto-positron",
    zoom=3,
    title="Walmart Store Sales by Geographic Location"
)
fig.show()


## ✅ Step 9: Monthly Forecasting Using Prophet

In [ ]:

df_weekly_sales['Month'] = df_weekly_sales['Date'].dt.to_period('M').dt.to_timestamp()
monthly_df = df_weekly_sales.groupby("Month").agg({"Total_Sales": "sum"}).reset_index()


In [ ]:

df_monthly_prophet = monthly_df.rename(columns={
    "Month": "ds",
    "Total_Sales": "y"
})


In [ ]:

!pip install prophet --quiet
from prophet import Prophet
import matplotlib.pyplot as plt


In [ ]:

model = Prophet()
model.fit(df_monthly_prophet)


In [ ]:

future = model.make_future_dataframe(periods=6, freq='M')
forecast = model.predict(future)


In [ ]:

fig = model.plot(forecast)
plt.title("Monthly Walmart Sales Forecast")
plt.xlabel("Month")
plt.ylabel("Sales")
plt.show()


In [ ]:

fig2 = model.plot_components(forecast)


## ✅ Step 10: Final Thoughts & Improvements